In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, VBox, Label, Layout
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, HuberRegressor

# Diagram settings
title_font_size = 12
axis_font_size = 12

# For reproducibility
np.random.seed(42)


def plot_description(text):
    print(f"\nDescription:\n{text}\n")


def linear_reg_loss_demo(num_outliers=50):
    # ---------------------------------------------------
    # 1. Create a simple regression dataset
    # ---------------------------------------------------
    n_samples = 500
    X, y = make_regression(n_samples=n_samples, n_features=1, noise=15, random_state=42)

    # Add outliers controlled by sliders
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 200 * np.random.randn(num_outliers)

    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    ols = LinearRegression().fit(X_scaled, y)

    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)

    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)

    # Predictions
    x_line = np.linspace(X_scaled.min(), X_scaled.max(), 100).reshape(-1, 1)
    y_ols = ols.predict(x_line)
    y_mae = sgd_mae.predict(x_line)
    y_huber = huber_reg.predict(x_line)
    
    # ---------------------------------------------------
    # 3. Plot results
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X_scaled, y, color='gray', alpha=0.4, label="Data")
    plt.plot(x_line, y_ols, label="MSE", linewidth=2)
    plt.plot(x_line, y_mae, label="MAE", linewidth=2)
    plt.plot(x_line, y_huber, label="Huber", linewidth=2)
    plt.title("Effect of Different Loss Functions on Fit", fontsize=title_font_size)
    plt.xlabel("X", fontsize=axis_font_size)
    plt.ylabel("y", fontsize=axis_font_size)
    plt.ylim([-600, 600])
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
    # 4. Create clearly labeled, wide sliders inside a titled block
    # ---------------------------------------------------
def linear_reg_loss_functions_demo_interact():
    plot_description("Three linear models using different loss functions (MSE/MAE/Huber) are fitted to the samples (gray dots). "
                     "Increase the number of outliers in the dataset and notice differences in the models. "
                     "Takeaway: Model predictions vary based on the loss function that was used to train the model.")

    num_outliers_slider = IntSlider(
        value=50, min=0, max=500, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        linear_reg_loss_demo,
        num_outliers=num_outliers_slider
    )

    display(ui_box, interactive_plot)